Data Generation

In [140]:
import numpy as np
from geomdl import BSpline
from geomdl import utilities
from geomdl import exchange
from geomdl.visualization import VisPlotly as vis
from geomdl import CPGen
from matplotlib import cm
import random

In [153]:
#Helper Functions

#from https://stackoverflow.com/questions/8487893/generate-all-the-points-on-the-circumference-of-a-circle
def PointsInCircum(r,z,n=15):
    return [[np.cos(2*np.pi/n*x)*r,np.sin(2*np.pi/n*x)*r, float(z)] for x in range(0,n)]

Generate Geometry

In [173]:
#Initialize vertices of bottle

numLevels = 5
numPtsOnLevel = 6
stepBetweenLevels = 1
radius = 2

#!!
numPoints = numLevels * numPtsOnLevel

offsetList = [random.random() for i in range(numPoints)]

mouthVertices = [PointsInCircum(3,0,numPtsOnLevel)]
topVertices = [PointsInCircum(radius,(numLevels + 1) * stepBetweenLevels,numPtsOnLevel)]

variableVertices = []
offsetListIndex = 0
for level in range(0 + 1, numLevels + 1):
    nextVertices = PointsInCircum(radius,level * stepBetweenLevels,numPtsOnLevel)
    for i in range(len(nextVertices) - 1):
        factor = offsetList[offsetListIndex]
        nextVertices[i][0] *= factor
        nextVertices[i][1] *= factor
        offsetListIndex += 1


    variableVertices += [nextVertices]

allVertices = mouthVertices + variableVertices + topVertices

print("Successfully generated points and a unique tensor!")


Successfully generated points and a unique tensor!


Preview Geometry

In [174]:
# Create a BSpline surface instance
surf = BSpline.Surface()

# Set degrees
surf.degree_u = 3
surf.degree_v = 3

# Get the control points from the generated grid
surf.ctrlpts2d = allVertices

# Set knot vectors
surf.knotvector_u = utilities.generate_knot_vector(surf.degree_u, surf.ctrlpts_size_u)
surf.knotvector_v = utilities.generate_knot_vector(surf.degree_v, surf.ctrlpts_size_v)

# Set sample size
surf.sample_size = 100

# Set visualization component
surf.vis = vis.VisSurface(ctrlpts=True, legend=False)
print("Generating visualization, please wait...")

# Plot the surface
surf.render(colormap=cm.terrain)

Generating visualization, please wait...


Save Geometry

In [180]:
stlName = ".\\stl\\example_stl.stl"
vecName = ".\\tensor\\example_tensor.txt"

exchange.export_stl(surf, stlName)
print("Surface successfully exported!\n...")

with open(vecName, "w") as output:
    output.write(str(offsetList))
print("Data successfully exported!")

Surface successfully exported!
...
Data successfully exported!
